# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 3 2022 11:42AM,238761,18,6541166,Hush Hush,Released
1,Jun 3 2022 11:35AM,238760,15,PNC363338A,"Person & Covey, Inc.",Released
2,Jun 3 2022 11:35AM,238760,15,PNC363340A,"Person & Covey, Inc.",Released
3,Jun 3 2022 11:35AM,238760,15,PNC363341A,"Person & Covey, Inc.",Released
4,Jun 3 2022 11:35AM,238758,15,8100084,"Brookfield Pharmaceuticals, LLC",Released
5,Jun 3 2022 11:34AM,238757,15,8096980,"Mizner Bioscience, LLC",Released
6,Jun 3 2022 11:34AM,238757,15,8096981,"Mizner Bioscience, LLC",Released
7,Jun 3 2022 11:34AM,238755,15,8100088,"Carwin Pharmaceutical Associates, LLC",Released
8,Jun 3 2022 11:33AM,238756,15,30009113,"Alliance Pharma, Inc.",Released
9,Jun 3 2022 11:33AM,238756,15,30009114,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,238756,Released,8
42,238757,Released,2
43,238758,Released,1
44,238760,Released,3
45,238761,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238756,NaN,NaN,8.0
238757,NaN,NaN,2.0
238758,NaN,NaN,1.0
238760,NaN,NaN,3.0
238761,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238654,0.0,0.0,1.0
238662,0.0,0.0,2.0
238670,3.0,8.0,1.0
238682,6.0,13.0,1.0
238710,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238654,0,0,1
238662,0,0,2
238670,3,8,1
238682,6,13,1
238710,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238654,0,0,1
1,238662,0,0,2
2,238670,3,8,1
3,238682,6,13,1
4,238710,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238654,,,1
1,238662,,,2
2,238670,3,8,1
3,238682,6,13,1
4,238710,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 3 2022 11:42AM,238761,18,Hush Hush
1,Jun 3 2022 11:35AM,238760,15,"Person & Covey, Inc."
4,Jun 3 2022 11:35AM,238758,15,"Brookfield Pharmaceuticals, LLC"
5,Jun 3 2022 11:34AM,238757,15,"Mizner Bioscience, LLC"
7,Jun 3 2022 11:34AM,238755,15,"Carwin Pharmaceutical Associates, LLC"
8,Jun 3 2022 11:33AM,238756,15,"Alliance Pharma, Inc."
16,Jun 3 2022 11:15AM,238733,15,"Virtus Pharmaceuticals, LLC"
34,Jun 3 2022 11:10AM,238753,10,Bio-PRF
35,Jun 3 2022 11:10AM,238752,10,Bio-PRF
36,Jun 3 2022 11:09AM,238751,10,Beach Patient Assistance


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 3 2022 11:42AM,238761,18,Hush Hush,,,1
1,Jun 3 2022 11:35AM,238760,15,"Person & Covey, Inc.",,,3
2,Jun 3 2022 11:35AM,238758,15,"Brookfield Pharmaceuticals, LLC",,,1
3,Jun 3 2022 11:34AM,238757,15,"Mizner Bioscience, LLC",,,2
4,Jun 3 2022 11:34AM,238755,15,"Carwin Pharmaceutical Associates, LLC",,,1
5,Jun 3 2022 11:33AM,238756,15,"Alliance Pharma, Inc.",,,8
6,Jun 3 2022 11:15AM,238733,15,"Virtus Pharmaceuticals, LLC",,,18
7,Jun 3 2022 11:10AM,238753,10,Bio-PRF,,,1
8,Jun 3 2022 11:10AM,238752,10,Bio-PRF,,,1
9,Jun 3 2022 11:09AM,238751,10,Beach Patient Assistance,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 11:42AM,238761,18,Hush Hush,1,,
1,Jun 3 2022 11:35AM,238760,15,"Person & Covey, Inc.",3,,
2,Jun 3 2022 11:35AM,238758,15,"Brookfield Pharmaceuticals, LLC",1,,
3,Jun 3 2022 11:34AM,238757,15,"Mizner Bioscience, LLC",2,,
4,Jun 3 2022 11:34AM,238755,15,"Carwin Pharmaceutical Associates, LLC",1,,
5,Jun 3 2022 11:33AM,238756,15,"Alliance Pharma, Inc.",8,,
6,Jun 3 2022 11:15AM,238733,15,"Virtus Pharmaceuticals, LLC",18,,
7,Jun 3 2022 11:10AM,238753,10,Bio-PRF,1,,
8,Jun 3 2022 11:10AM,238752,10,Bio-PRF,1,,
9,Jun 3 2022 11:09AM,238751,10,Beach Patient Assistance,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 11:42AM,238761,18,Hush Hush,1,,
1,Jun 3 2022 11:35AM,238760,15,"Person & Covey, Inc.",3,,
2,Jun 3 2022 11:35AM,238758,15,"Brookfield Pharmaceuticals, LLC",1,,
3,Jun 3 2022 11:34AM,238757,15,"Mizner Bioscience, LLC",2,,
4,Jun 3 2022 11:34AM,238755,15,"Carwin Pharmaceutical Associates, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 11:42AM,238761,18,Hush Hush,1.0,NaN,NaN
1,Jun 3 2022 11:35AM,238760,15,"Person & Covey, Inc.",3.0,NaN,NaN
2,Jun 3 2022 11:35AM,238758,15,"Brookfield Pharmaceuticals, LLC",1.0,NaN,NaN
3,Jun 3 2022 11:34AM,238757,15,"Mizner Bioscience, LLC",2.0,NaN,NaN
4,Jun 3 2022 11:34AM,238755,15,"Carwin Pharmaceutical Associates, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 11:42AM,238761,18,Hush Hush,1.0,0.0,0.0
1,Jun 3 2022 11:35AM,238760,15,"Person & Covey, Inc.",3.0,0.0,0.0
2,Jun 3 2022 11:35AM,238758,15,"Brookfield Pharmaceuticals, LLC",1.0,0.0,0.0
3,Jun 3 2022 11:34AM,238757,15,"Mizner Bioscience, LLC",2.0,0.0,0.0
4,Jun 3 2022 11:34AM,238755,15,"Carwin Pharmaceutical Associates, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4058365,100.0,26.0,4.0
12,716176,12.0,14.0,0.0
15,1432519,33.0,0.0,0.0
16,716147,3.0,0.0,0.0
18,716202,3.0,0.0,0.0
19,477467,11.0,0.0,1.0
20,716145,11.0,22.0,6.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4058365,100.0,26.0,4.0
1,12,716176,12.0,14.0,0.0
2,15,1432519,33.0,0.0,0.0
3,16,716147,3.0,0.0,0.0
4,18,716202,3.0,0.0,0.0
5,19,477467,11.0,0.0,1.0
6,20,716145,11.0,22.0,6.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,100.0,26.0,4.0
1,12,12.0,14.0,0.0
2,15,33.0,0.0,0.0
3,16,3.0,0.0,0.0
4,18,3.0,0.0,0.0
5,19,11.0,0.0,1.0
6,20,11.0,22.0,6.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,100.0
1,12,Released,12.0
2,15,Released,33.0
3,16,Released,3.0
4,18,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20
Status,,,,,,,
Completed,4.0,0.0,0.0,0.0,0.0,1.0,6.0
Executing,26.0,14.0,0.0,0.0,0.0,0.0,22.0
Released,100.0,12.0,33.0,3.0,3.0,11.0,11.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20
0,Completed,4.0,0.0,0.0,0.0,0.0,1.0,6.0
1,Executing,26.0,14.0,0.0,0.0,0.0,0.0,22.0
2,Released,100.0,12.0,33.0,3.0,3.0,11.0,11.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20
0,Completed,4.0,0.0,0.0,0.0,0.0,1.0,6.0
1,Executing,26.0,14.0,0.0,0.0,0.0,0.0,22.0
2,Released,100.0,12.0,33.0,3.0,3.0,11.0,11.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()